# Purpose
Predict survivability for titanic passengers

In [1]:
from sklearn import tree
from sklearn import model_selection
from sklearn import metrics

from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model #SGDClassifier, LogisticRegression, Perceptron
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB

from xgboost import XGBClassifier
from xgboost import XGBRegressor
from xgboost import XGBRFClassifier
from xgboost import XGBRFRegressor

import numpy as np
import matplotlib as plt
import seaborn as sns
import pandas as pd

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [3]:
filepath = 'C:/Users/nowic/Desktop/home_tasks/pets/titanic/feather_data/'

In [4]:
train_mean = pd.read_feather(f'{filepath}train_mean_age.feather')
train_median = pd.read_feather(f'{filepath}train_median_age.feather')

test_mean = pd.read_feather(f'{filepath}test_mean_age.feather')
test_median = pd.read_feather(f'{filepath}test_median_age.feather')

In [ ]:
# X_test['Fare'] = X_test['Age'].fillna(
#     X_test.query('Pclass == 3')['Fare'].mode()) # NaN in 3rd class passenger's fare

# train_mean

In [5]:
train_mean

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Age_class
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,Young
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,Middle-aged
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,Young
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,Middle-aged
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,Middle-aged
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,S,Young
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,S,Young
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,30.0,1,2,W./C. 6607,23.4500,S,Young
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C,Young


## With Age column

### With Fare

In [6]:
y_train = train_mean['Survived']
y_train

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [25]:
X_train_age_fare = train_mean.drop(['PassengerId', 'Survived', 'Age_class', 'Name', 'Ticket'], axis=1)

X_train_age_fare

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...
886,2,male,27.0,0,0,13.0000,S
887,1,female,19.0,0,0,30.0000,S
888,3,female,30.0,1,2,23.4500,S
889,1,male,26.0,0,0,30.0000,C


In [26]:
X_train_age_fare = pd.get_dummies(X_train_age_fare)
X_train_age_fare

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,22.0,1,0,7.2500,0,1,0,0,1
1,1,38.0,1,0,71.2833,1,0,1,0,0
2,3,26.0,0,0,7.9250,1,0,0,0,1
3,1,35.0,1,0,53.1000,1,0,0,0,1
4,3,35.0,0,0,8.0500,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...
886,2,27.0,0,0,13.0000,0,1,0,0,1
887,1,19.0,0,0,30.0000,1,0,0,0,1
888,3,30.0,1,2,23.4500,1,0,0,0,1
889,1,26.0,0,0,30.0000,0,1,1,0,0


#### DecisionTreeClassifier

In [104]:
decision_tree = tree.DecisionTreeClassifier()
params = {'criterion': ['gini', 'entropy'],
          'max_depth': range(1, 31),
          'min_samples_leaf': range(1, 21)}

grid_search = model_selection.GridSearchCV(decision_tree, param_grid=params, cv=5, n_jobs=-1)
grid_search.fit(X_train_age_fare, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': range(1, 31),
                         'min_samples_leaf': range(1, 21)})

In [105]:
grid_search.best_params_, grid_search.best_score_

({'criterion': 'gini', 'max_depth': 7, 'min_samples_leaf': 6},
 0.8316678174628084)

#### RandomForestClassifier

In [106]:
random_forest = RandomForestClassifier()
params = {'criterion': ['gini', 'entropy'],
          'max_depth': range(1, 31),
          'min_samples_leaf': range(1, 21),
          'n_estimators': range(1, 11)}

grid_search = model_selection.GridSearchCV(random_forest, param_grid=params, cv=5, n_jobs=-1)
grid_search.fit(X_train_age_fare, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': range(1, 31),
                         'min_samples_leaf': range(1, 21),
                         'n_estimators': range(1, 11)})

In [107]:
grid_search.best_params_, grid_search.best_score_

({'criterion': 'gini',
  'max_depth': 27,
  'min_samples_leaf': 2,
  'n_estimators': 10},
 0.8372669637813068)

#### KNeighborsClassifier

In [108]:
k_neighbors = KNeighborsClassifier()
params = {'n_neighbors': range(5, 15),
          'weights': ['uniform', 'distance'],
          'algorithm': ['ball_tree', 'kd_tree', 'brute']}

grid_search = model_selection.GridSearchCV(k_neighbors, param_grid=params, cv=5, n_jobs=-1)
grid_search.fit(X_train_age_fare, y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'algorithm': ['ball_tree', 'kd_tree', 'brute'],
                         'n_neighbors': range(5, 15),
                         'weights': ['uniform', 'distance']})

In [109]:
grid_search.best_params_, grid_search.best_score_

({'algorithm': 'brute', 'n_neighbors': 6, 'weights': 'distance'},
 0.7306948716339212)

#### Perceptron

In [110]:
perceptron = linear_model.Perceptron()
params = {'penalty': ['l1', 'l2', 'elasticnet'],
          'alpha': np.arange(0.0001, 0.0005, 0.0001)}

grid_search = model_selection.GridSearchCV(perceptron, param_grid=params, cv=5, n_jobs=-1)
grid_search.fit(X_train_age_fare, y_train)

GridSearchCV(cv=5, estimator=Perceptron(), n_jobs=-1,
             param_grid={'alpha': array([0.0001, 0.0002, 0.0003, 0.0004]),
                         'penalty': ['l1', 'l2', 'elasticnet']})

In [111]:
grid_search.best_params_, grid_search.best_score_

({'alpha': 0.0001, 'penalty': 'l1'}, 0.7263134768689976)

#### LogisticRegression

In [112]:
logistic_regression = linear_model.LogisticRegression()
params = {'penalty': ['l1', 'l2', 'elasticnet', 'none'],
          'dual': [True, False],
          'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
          'multi_class': ['ovr', 'multinomial']}

grid_search = model_selection.GridSearchCV(logistic_regression, param_grid=params, cv=5, n_jobs=-1)
grid_search.fit(X_train_age_fare, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
295 fits failed out of a total of 400.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 447, in _check_solver
    raise ValueError(
Va

GridSearchCV(cv=5, estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'dual': [True, False],
                         'multi_class': ['ovr', 'multinomial'],
                         'penalty': ['l1', 'l2', 'elasticnet', 'none'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                                    'saga']})

In [113]:
grid_search.best_params_, grid_search.best_score_

({'dual': False,
  'multi_class': 'multinomial',
  'penalty': 'none',
  'solver': 'lbfgs'},
 0.7912623187496077)

#### SGDClassifier

In [114]:
sgd_classifier = linear_model.SGDClassifier()
params = {'loss': ['hinge', 'squared_hinge', 'log', 'squared_error'],
          'penalty': ['l1', 'l2', 'elasticnet']}

grid_search = model_selection.GridSearchCV(sgd_classifier, param_grid=params, cv=5, n_jobs=-1)
grid_search.fit(X_train_age_fare, y_train)

GridSearchCV(cv=5, estimator=SGDClassifier(), n_jobs=-1,
             param_grid={'loss': ['hinge', 'squared_hinge', 'log',
                                  'squared_error'],
                         'penalty': ['l1', 'l2', 'elasticnet']})

In [115]:
grid_search.best_params_, grid_search.best_score_

({'loss': 'log', 'penalty': 'l1'}, 0.7721674722239659)

#### LinearSVC

In [116]:
linear_svc = LinearSVC()
params = {'penalty': ['l1', 'l2'],
          'loss': ['hinge', 'squared_hinge'],
          'dual': [True, False],
          'multi_class': ['ovr', 'crammer_singer'],
          'class_weight': [None, 'balanced']}

grid_search = model_selection.GridSearchCV(linear_svc, param_grid=params, cv=5, n_jobs=-1)
grid_search.fit(X_train_age_fare, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
40 fits failed out of a total of 160.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_classes.py", line 257, in fit
    self.coef_, self.intercept_, self.n_iter_ = _fit_liblinear(
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py", line 1185, in _fit_liblinear
    solver_type = _get_liblinear_solver_type(mul

GridSearchCV(cv=5, estimator=LinearSVC(), n_jobs=-1,
             param_grid={'class_weight': [None, 'balanced'],
                         'dual': [True, False],
                         'loss': ['hinge', 'squared_hinge'],
                         'multi_class': ['ovr', 'crammer_singer'],
                         'penalty': ['l1', 'l2']})

In [117]:
grid_search.best_params_, grid_search.best_score_

({'class_weight': None,
  'dual': False,
  'loss': 'squared_hinge',
  'multi_class': 'ovr',
  'penalty': 'l1'},
 0.7901261691042621)

#### GaussianNB

In [118]:
gaussian = GaussianNB()
gaussian.fit(X_train_age_fare, y_train)
gaussian.score(X_train_age_fare, y_train)

0.7912457912457912

#### XGBClassifier

In [27]:
xgb_cl = XGBClassifier()
params = {'n_estimators': range(1, 11),
          'max_depth': range(1, 31),
          'learning_rate': np.arange(0.1, 1, 0.1),
          'verbosity': range(1, 4),
          'booster': ['gbtree', 'gblinear', 'dart']}

grid_search = model_selection.GridSearchCV(xgb_cl, param_grid=params, cv=5, n_jobs=-1)
grid_search.fit(X_train_age_fare, y_train)

[20:37:32] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     enable_categorical=False, gamma=None,
                                     gpu_id=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,...
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, reg_alpha=None,
                                     reg_lambda=None, scale_pos_weight=None,
                                     subsample=None, tree_

In [28]:
grid_search.best_params_, grid_search.best_score_

({'booster': 'gbtree',
  'learning_rate': 0.4,
  'max_depth': 8,
  'n_estimators': 8,
  'verbosity': 1},
 0.8417801770133702)

#### XGBRFClassifier

In [29]:
xgbrf_cl = XGBRFClassifier()
params = {'n_estimators': range(1, 11),
          'max_depth': range(1, 31),
          'learning_rate': np.arange(0.1, 1, 0.1),
          'verbosity': range(1, 4),
          'booster': ['gbtree', 'gblinear', 'dart']}

grid_search = model_selection.GridSearchCV(xgbrf_cl, param_grid=params, cv=5, n_jobs=-1)
grid_search.fit(X_train_age_fare, y_train)

[20:52:35] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


GridSearchCV(cv=5,
             estimator=XGBRFClassifier(base_score=None, booster=None,
                                       colsample_bylevel=None,
                                       colsample_bytree=None,
                                       enable_categorical=False, gamma=None,
                                       gpu_id=None, importance_type=None,
                                       interaction_constraints=None,
                                       max_delta_step=None, max_depth=None,
                                       min_child_weight=None, missing=nan,
                                       monotone_constraints=None,
                                       n_estimators=100, n_jobs=None,
                                       num_parall...
                                       objective='binary:logistic',
                                       predictor=None, random_state=None,
                                       reg_alpha=None, scale_pos_weight=None,
      

In [30]:
grid_search.best_params_, grid_search.best_score_

({'booster': 'gbtree',
  'learning_rate': 0.1,
  'max_depth': 13,
  'n_estimators': 6,
  'verbosity': 1},
 0.8384156675663801)

### W/o Fare

In [33]:
X_train_age = train_mean.drop(['PassengerId', 'Survived', 'Age_class', 'Name', 'Ticket', 'Fare'], axis=1)
X_train_age

,Pclass,Sex,Age,SibSp,Parch,Embarked
0,3,male,22.0,1,0,S
1,1,female,38.0,1,0,C
2,3,female,26.0,0,0,S
3,1,female,35.0,1,0,S
4,3,male,35.0,0,0,S
...,...,...,...,...,...,...
886,2,male,27.0,0,0,S
887,1,female,19.0,0,0,S
888,3,female,30.0,1,2,S
889,1,male,26.0,0,0,C


In [34]:
X_train_age = pd.get_dummies(X_train_age)
X_train_age

,Pclass,Age,SibSp,Parch,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,22.0,1,0,0,1,0,0,1
1,1,38.0,1,0,1,0,1,0,0
2,3,26.0,0,0,1,0,0,0,1
3,1,35.0,1,0,1,0,0,0,1
4,3,35.0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...
886,2,27.0,0,0,0,1,0,0,1
887,1,19.0,0,0,1,0,0,0,1
888,3,30.0,1,2,1,0,0,0,1
889,1,26.0,0,0,0,1,1,0,0


#### DecisionTreeClassifier

In [121]:
decision_tree = tree.DecisionTreeClassifier()
params = {'criterion': ['gini', 'entropy'],
          'max_depth': range(1, 31),
          'min_samples_leaf': range(1, 21)}

grid_search = model_selection.GridSearchCV(decision_tree, param_grid=params, cv=5, n_jobs=-1)
grid_search.fit(X_train_age, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': range(1, 31),
                         'min_samples_leaf': range(1, 21)})

In [122]:
grid_search.best_params_, grid_search.best_score_

({'criterion': 'gini', 'max_depth': 9, 'min_samples_leaf': 8},
 0.8193773146695122)

#### RandomForestClassifier

In [123]:
random_forest = RandomForestClassifier()
params = {'criterion': ['gini', 'entropy'],
          'max_depth': range(1, 31),
          'min_samples_leaf': range(1, 21),
          'n_estimators': range(1, 11)}

grid_search = model_selection.GridSearchCV(random_forest, param_grid=params, cv=5, n_jobs=-1)
grid_search.fit(X_train_age, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': range(1, 31),
                         'min_samples_leaf': range(1, 21),
                         'n_estimators': range(1, 11)})

In [124]:
grid_search.best_params_, grid_search.best_score_

({'criterion': 'entropy',
  'max_depth': 9,
  'min_samples_leaf': 5,
  'n_estimators': 7},
 0.8316615403929445)

#### KNeighborsClassifier

In [125]:
from sklearn.neighbors import KNeighborsClassifier

k_neighbors = KNeighborsClassifier()
params = {'n_neighbors': range(5, 15),
          'weights': ['uniform', 'distance'],
          'algorithm': ['ball_tree', 'kd_tree', 'brute']}

grid_search = model_selection.GridSearchCV(k_neighbors, param_grid=params, cv=5, n_jobs=-1)
grid_search.fit(X_train_age, y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'algorithm': ['ball_tree', 'kd_tree', 'brute'],
                         'n_neighbors': range(5, 15),
                         'weights': ['uniform', 'distance']})

In [126]:
grid_search.best_params_, grid_search.best_score_

({'algorithm': 'ball_tree', 'n_neighbors': 13, 'weights': 'distance'},
 0.8069926558282594)

#### Perceptron

In [127]:
perceptron = linear_model.Perceptron()
params = {'penalty': ['l1', 'l2', 'elasticnet'],
          'alpha': np.arange(0.0001, 0.0005, 0.0001)}

grid_search = model_selection.GridSearchCV(perceptron, param_grid=params, cv=5, n_jobs=-1)
grid_search.fit(X_train_age, y_train)

GridSearchCV(cv=5, estimator=Perceptron(), n_jobs=-1,
             param_grid={'alpha': array([0.0001, 0.0002, 0.0003, 0.0004]),
                         'penalty': ['l1', 'l2', 'elasticnet']})

In [128]:
perceptron_estimator = grid_search.best_estimator_
grid_search.best_params_, grid_search.best_score_

({'alpha': 0.0002, 'penalty': 'l1'}, 0.7206201745025422)

#### LogisticRegression

In [129]:
logistic_regression = linear_model.LogisticRegression()
params = {'penalty': ['l1', 'l2', 'elasticnet', 'none'],
          'dual': [True, False],
          'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
          'multi_class': ['ovr', 'multinomial']}

grid_search = model_selection.GridSearchCV(logistic_regression, param_grid=params, cv=5, n_jobs=-1)
grid_search.fit(X_train_age, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
295 fits failed out of a total of 400.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 447, in _check_solver
    raise ValueError(
Va

GridSearchCV(cv=5, estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'dual': [True, False],
                         'multi_class': ['ovr', 'multinomial'],
                         'penalty': ['l1', 'l2', 'elasticnet', 'none'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                                    'saga']})

In [130]:
grid_search.best_params_, grid_search.best_score_

({'dual': False, 'multi_class': 'ovr', 'penalty': 'l1', 'solver': 'saga'},
 0.794589165777415)

#### SGDClassifier

In [131]:
sgd_classifier = linear_model.SGDClassifier()
params = {'loss': ['hinge', 'squared_hinge', 'log', 'squared_error'],
          'penalty': ['l1', 'l2', 'elasticnet']}

grid_search = model_selection.GridSearchCV(sgd_classifier, param_grid=params, cv=5, n_jobs=-1)
grid_search.fit(X_train_age, y_train)

GridSearchCV(cv=5, estimator=SGDClassifier(), n_jobs=-1,
             param_grid={'loss': ['hinge', 'squared_hinge', 'log',
                                  'squared_error'],
                         'penalty': ['l1', 'l2', 'elasticnet']})

In [132]:
grid_search.best_params_, grid_search.best_score_

({'loss': 'squared_hinge', 'penalty': 'elasticnet'}, 0.7924235766744083)

#### LinearSVC

In [133]:
linear_svc = LinearSVC()
params = {'penalty': ['l1', 'l2'],
          'loss': ['hinge', 'squared_hinge'],
          'dual': [True, False],
          'multi_class': ['ovr', 'crammer_singer'],
          'class_weight': [None, 'balanced']}

grid_search = model_selection.GridSearchCV(linear_svc, param_grid=params, cv=5, n_jobs=-1)
grid_search.fit(X_train_age, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
40 fits failed out of a total of 160.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_classes.py", line 257, in fit
    self.coef_, self.intercept_, self.n_iter_ = _fit_liblinear(
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py", line 1185, in _fit_liblinear
    solver_type = _get_liblinear_solver_type(mul

GridSearchCV(cv=5, estimator=LinearSVC(), n_jobs=-1,
             param_grid={'class_weight': [None, 'balanced'],
                         'dual': [True, False],
                         'loss': ['hinge', 'squared_hinge'],
                         'multi_class': ['ovr', 'crammer_singer'],
                         'penalty': ['l1', 'l2']})

In [134]:
grid_search.best_params_, grid_search.best_score_

({'class_weight': None,
  'dual': False,
  'loss': 'squared_hinge',
  'multi_class': 'ovr',
  'penalty': 'l1'},
 0.789008850668508)

#### GaussianNB

In [136]:
gaussian = GaussianNB()
gaussian.fit(X_train_age, y_train)
gaussian.score(X_train_age, y_train)

0.7912457912457912

#### XGBClassifier

In [35]:
xgb_cl = XGBClassifier()
params = {'n_estimators': range(1, 11),
          'max_depth': range(1, 31),
          'learning_rate': np.arange(0.1, 1, 0.1),
          'verbosity': range(1, 4),
          'booster': ['gbtree', 'gblinear', 'dart']}

grid_search = model_selection.GridSearchCV(xgb_cl, param_grid=params, cv=5, n_jobs=-1)
grid_search.fit(X_train_age, y_train)

[21:10:14] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     enable_categorical=False, gamma=None,
                                     gpu_id=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,...
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, reg_alpha=None,
                                     reg_lambda=None, scale_pos_weight=None,
                                     subsample=None, tree_

In [36]:
grid_search.best_params_, grid_search.best_score_

({'booster': 'gbtree',
  'learning_rate': 0.8,
  'max_depth': 5,
  'n_estimators': 7,
  'verbosity': 1},
 0.8395204318624065)

#### XGBRFClassifier

In [37]:
xgbrf_cl = XGBRFClassifier()
params = {'n_estimators': range(1, 11),
          'max_depth': range(1, 31),
          'learning_rate': np.arange(0.1, 1, 0.1),
          'verbosity': range(1, 4),
          'booster': ['gbtree', 'gblinear', 'dart']}

grid_search = model_selection.GridSearchCV(xgbrf_cl, param_grid=params, cv=5, n_jobs=-1)
grid_search.fit(X_train_age, y_train)

[21:25:40] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


GridSearchCV(cv=5,
             estimator=XGBRFClassifier(base_score=None, booster=None,
                                       colsample_bylevel=None,
                                       colsample_bytree=None,
                                       enable_categorical=False, gamma=None,
                                       gpu_id=None, importance_type=None,
                                       interaction_constraints=None,
                                       max_delta_step=None, max_depth=None,
                                       min_child_weight=None, missing=nan,
                                       monotone_constraints=None,
                                       n_estimators=100, n_jobs=None,
                                       num_parall...
                                       objective='binary:logistic',
                                       predictor=None, random_state=None,
                                       reg_alpha=None, scale_pos_weight=None,
      

In [38]:
grid_search.best_params_, grid_search.best_score_

({'booster': 'gbtree',
  'learning_rate': 0.1,
  'max_depth': 9,
  'n_estimators': 6,
  'verbosity': 1},
 0.8227229929069111)

## With Age_class

### With Fare

In [44]:
X_train_ageclass_fare = train_mean.drop(['PassengerId', 'Survived', 'Age', 'Name', 'Ticket'], axis=1)

In [138]:
X_train_ageclass_fare

,Pclass,Sex,SibSp,Parch,Fare,Embarked,Age_class
0,3,male,1,0,7.2500,S,Young
1,1,female,1,0,71.2833,C,Middle-aged
2,3,female,0,0,7.9250,S,Young
3,1,female,1,0,53.1000,S,Middle-aged
4,3,male,0,0,8.0500,S,Middle-aged
...,...,...,...,...,...,...,...
886,2,male,0,0,13.0000,S,Young
887,1,female,0,0,30.0000,S,Young
888,3,female,1,2,23.4500,S,Young
889,1,male,0,0,30.0000,C,Young


In [45]:
X_train_ageclass_fare = pd.get_dummies(X_train_ageclass_fare)
X_train_ageclass_fare

,Pclass,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Age_class_Child,Age_class_Middle-aged,Age_class_Old-aged,Age_class_Young
0,3,1,0,7.2500,0,1,0,0,1,0,0,0,1
1,1,1,0,71.2833,1,0,1,0,0,0,1,0,0
2,3,0,0,7.9250,1,0,0,0,1,0,0,0,1
3,1,1,0,53.1000,1,0,0,0,1,0,1,0,0
4,3,0,0,8.0500,0,1,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,2,0,0,13.0000,0,1,0,0,1,0,0,0,1
887,1,0,0,30.0000,1,0,0,0,1,0,0,0,1
888,3,1,2,23.4500,1,0,0,0,1,0,0,0,1
889,1,0,0,30.0000,0,1,1,0,0,0,0,0,1


#### DecisionTreeClassifier

In [140]:
decision_tree = tree.DecisionTreeClassifier()
params = {'criterion': ['gini', 'entropy'],
          'max_depth': range(1, 31),
          'min_samples_leaf': range(1, 21)}

grid_search = model_selection.GridSearchCV(decision_tree, param_grid=params, cv=5, n_jobs=-1)
grid_search.fit(X_train_ageclass_fare, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': range(1, 31),
                         'min_samples_leaf': range(1, 21)})

In [141]:
grid_search.best_params_, grid_search.best_score_

({'criterion': 'entropy', 'max_depth': 7, 'min_samples_leaf': 3},
 0.8260561170045824)

#### RandomForestClassifier

In [142]:
random_forest = RandomForestClassifier()
params = {'criterion': ['gini', 'entropy'],
          'max_depth': range(1, 31),
          'min_samples_leaf': range(1, 21),
          'n_estimators': range(1, 11)}

grid_search = model_selection.GridSearchCV(random_forest, param_grid=params, cv=5, n_jobs=-1)
grid_search.fit(X_train_ageclass_fare, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': range(1, 31),
                         'min_samples_leaf': range(1, 21),
                         'n_estimators': range(1, 11)})

In [143]:
grid_search.best_params_, grid_search.best_score_

({'criterion': 'entropy',
  'max_depth': 19,
  'min_samples_leaf': 2,
  'n_estimators': 8},
 0.8350260498399347)

#### KNeighborsClassifier

In [144]:
k_neighbors = KNeighborsClassifier()
params = {'n_neighbors': range(5, 15),
          'weights': ['uniform', 'distance'],
          'algorithm': ['ball_tree', 'kd_tree', 'brute']}

grid_search = model_selection.GridSearchCV(k_neighbors, param_grid=params, cv=5, n_jobs=-1)
grid_search.fit(X_train_ageclass_fare, y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'algorithm': ['ball_tree', 'kd_tree', 'brute'],
                         'n_neighbors': range(5, 15),
                         'weights': ['uniform', 'distance']})

In [145]:
grid_search.best_params_, grid_search.best_score_

({'algorithm': 'brute', 'n_neighbors': 5, 'weights': 'distance'},
 0.7766744083861653)

#### Perceptron

In [146]:
perceptron = linear_model.Perceptron()
params = {'penalty': ['l1', 'l2', 'elasticnet'],
          'alpha': np.arange(0.0001, 0.0005, 0.0001)}

grid_search = model_selection.GridSearchCV(perceptron, param_grid=params, cv=5, n_jobs=-1)
grid_search.fit(X_train_ageclass_fare, y_train)

GridSearchCV(cv=5, estimator=Perceptron(), n_jobs=-1,
             param_grid={'alpha': array([0.0001, 0.0002, 0.0003, 0.0004]),
                         'penalty': ['l1', 'l2', 'elasticnet']})

In [147]:
grid_search.best_params_, grid_search.best_score_

({'alpha': 0.0002, 'penalty': 'l1'}, 0.735107651748164)

#### LogisticRegression

In [148]:
logistic_regression = linear_model.LogisticRegression()
params = {'penalty': ['l1', 'l2', 'elasticnet', 'none'],
          'dual': [True, False],
          'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
          'multi_class': ['ovr', 'multinomial']}

grid_search = model_selection.GridSearchCV(logistic_regression, param_grid=params, cv=5, n_jobs=-1)
grid_search.fit(X_train_ageclass_fare, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
295 fits failed out of a total of 400.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 447, in _check_solver
    raise ValueError(
Va

GridSearchCV(cv=5, estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'dual': [True, False],
                         'multi_class': ['ovr', 'multinomial'],
                         'penalty': ['l1', 'l2', 'elasticnet', 'none'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                                    'saga']})

In [149]:
grid_search.best_params_, grid_search.best_score_

({'dual': False, 'multi_class': 'ovr', 'penalty': 'l1', 'solver': 'liblinear'},
 0.7856506182913815)

#### SGDClassifier

In [150]:
sgd_classifier = linear_model.SGDClassifier()
params = {'loss': ['hinge', 'squared_hinge', 'log', 'squared_error'],
          'penalty': ['l1', 'l2', 'elasticnet']}

grid_search = model_selection.GridSearchCV(sgd_classifier, param_grid=params, cv=5, n_jobs=-1)
grid_search.fit(X_train_ageclass_fare, y_train)

GridSearchCV(cv=5, estimator=SGDClassifier(), n_jobs=-1,
             param_grid={'loss': ['hinge', 'squared_hinge', 'log',
                                  'squared_error'],
                         'penalty': ['l1', 'l2', 'elasticnet']})

In [151]:
grid_search.best_params_, grid_search.best_score_

({'loss': 'log', 'penalty': 'elasticnet'}, 0.7755570899504111)

#### LinearSVC

In [152]:
linear_svc = LinearSVC()
params = {'penalty': ['l1', 'l2'],
          'loss': ['hinge', 'squared_hinge'],
          'dual': [True, False],
          'multi_class': ['ovr', 'crammer_singer'],
          'class_weight': [None, 'balanced']}

grid_search = model_selection.GridSearchCV(linear_svc, param_grid=params, cv=5, n_jobs=-1)
grid_search.fit(X_train_ageclass_fare, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
40 fits failed out of a total of 160.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_classes.py", line 257, in fit
    self.coef_, self.intercept_, self.n_iter_ = _fit_liblinear(
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py", line 1185, in _fit_liblinear
    solver_type = _get_liblinear_solver_type(mul

GridSearchCV(cv=5, estimator=LinearSVC(), n_jobs=-1,
             param_grid={'class_weight': [None, 'balanced'],
                         'dual': [True, False],
                         'loss': ['hinge', 'squared_hinge'],
                         'multi_class': ['ovr', 'crammer_singer'],
                         'penalty': ['l1', 'l2']})

In [153]:
grid_search.best_params_, grid_search.best_score_

({'class_weight': 'balanced',
  'dual': True,
  'loss': 'hinge',
  'multi_class': 'crammer_singer',
  'penalty': 'l1'},
 0.792348251836043)

#### GaussianNB

In [154]:
gaussian = GaussianNB()
gaussian.fit(X_train_ageclass_fare, y_train)
gaussian.score(X_train_ageclass_fare, y_train)

0.7946127946127947

#### XGBClassifier

In [46]:
xgb_cl = XGBClassifier()
params = {'n_estimators': range(1, 11),
          'max_depth': range(1, 31),
          'learning_rate': np.arange(0.1, 1, 0.1),
          'verbosity': range(1, 4),
          'booster': ['gbtree', 'gblinear', 'dart']}

grid_search = model_selection.GridSearchCV(xgb_cl, param_grid=params, cv=5, n_jobs=-1)
grid_search.fit(X_train_ageclass_fare, y_train)

[22:00:52] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     enable_categorical=False, gamma=None,
                                     gpu_id=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,...
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, reg_alpha=None,
                                     reg_lambda=None, scale_pos_weight=None,
                                     subsample=None, tree_

In [47]:
grid_search.best_params_, grid_search.best_score_

({'booster': 'gbtree',
  'learning_rate': 0.30000000000000004,
  'max_depth': 12,
  'n_estimators': 6,
  'verbosity': 1},
 0.8361684765551439)

#### XGBRFClassifier

In [49]:
xgbrf_cl = XGBRFClassifier()
params = {'n_estimators': range(1, 11),
          'max_depth': range(1, 31),
          'learning_rate': np.arange(0.1, 1, 0.1),
          'verbosity': range(1, 4),
          'booster': ['gbtree', 'gblinear', 'dart']}

grid_search = model_selection.GridSearchCV(xgbrf_cl, param_grid=params, cv=5, n_jobs=-1)
grid_search.fit(X_train_ageclass_fare, y_train)

[22:13:02] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


GridSearchCV(cv=5,
             estimator=XGBRFClassifier(base_score=None, booster=None,
                                       colsample_bylevel=None,
                                       colsample_bytree=None,
                                       enable_categorical=False, gamma=None,
                                       gpu_id=None, importance_type=None,
                                       interaction_constraints=None,
                                       max_delta_step=None, max_depth=None,
                                       min_child_weight=None, missing=nan,
                                       monotone_constraints=None,
                                       n_estimators=100, n_jobs=None,
                                       num_parall...
                                       objective='binary:logistic',
                                       predictor=None, random_state=None,
                                       reg_alpha=None, scale_pos_weight=None,
      

In [51]:
grid_search.best_params_, grid_search.best_score_

({'booster': 'gbtree',
  'learning_rate': 0.1,
  'max_depth': 9,
  'n_estimators': 5,
  'verbosity': 1},
 0.8283095850856819)

### W/o Fare

In [7]:
X_train_ageclass = train_mean.drop(['PassengerId', 'Survived', 'Age', 'Name', 'Ticket', 'Fare'], axis=1)
X_train_ageclass

,Pclass,Sex,SibSp,Parch,Embarked,Age_class
0,3,male,1,0,S,Young
1,1,female,1,0,C,Middle-aged
2,3,female,0,0,S,Young
3,1,female,1,0,S,Middle-aged
4,3,male,0,0,S,Middle-aged
...,...,...,...,...,...,...
886,2,male,0,0,S,Young
887,1,female,0,0,S,Young
888,3,female,1,2,S,Young
889,1,male,0,0,C,Young


In [8]:
X_train_ageclass = pd.get_dummies(X_train_ageclass)
X_train_ageclass

,Pclass,SibSp,Parch,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Age_class_Child,Age_class_Middle-aged,Age_class_Old-aged,Age_class_Young
0,3,1,0,0,1,0,0,1,0,0,0,1
1,1,1,0,1,0,1,0,0,0,1,0,0
2,3,0,0,1,0,0,0,1,0,0,0,1
3,1,1,0,1,0,0,0,1,0,1,0,0
4,3,0,0,0,1,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
886,2,0,0,0,1,0,0,1,0,0,0,1
887,1,0,0,1,0,0,0,1,0,0,0,1
888,3,1,2,1,0,0,0,1,0,0,0,1
889,1,0,0,0,1,1,0,0,0,0,0,1


#### DecisionTreeClassifier

In [157]:
decision_tree = tree.DecisionTreeClassifier()
params = {'criterion': ['gini', 'entropy'],
          'max_depth': range(1, 31),
          'min_samples_leaf': range(1, 21)}

grid_search = model_selection.GridSearchCV(decision_tree, param_grid=params, cv=5, n_jobs=-1)
grid_search.fit(X_train_ageclass, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': range(1, 31),
                         'min_samples_leaf': range(1, 21)})

In [158]:
grid_search.best_params_, grid_search.best_score_

({'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 3},
 0.8092021844203126)

#### RandomForestClassifier

In [159]:
random_forest = RandomForestClassifier()
params = {'criterion': ['gini', 'entropy'],
          'max_depth': range(1, 31),
          'min_samples_leaf': range(1, 21),
          'n_estimators': range(1, 11)}

grid_search = model_selection.GridSearchCV(random_forest, param_grid=params, cv=5, n_jobs=-1)
grid_search.fit(X_train_ageclass, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': range(1, 31),
                         'min_samples_leaf': range(1, 21),
                         'n_estimators': range(1, 11)})

In [160]:
grid_search.best_params_, grid_search.best_score_

({'criterion': 'gini',
  'max_depth': 25,
  'min_samples_leaf': 4,
  'n_estimators': 6},
 0.8283033080158182)

#### KNeighborsClassifier

In [161]:
k_neighbors = KNeighborsClassifier()
params = {'n_neighbors': range(5, 15),
          'weights': ['uniform', 'distance'],
          'algorithm': ['ball_tree', 'kd_tree', 'brute']}

grid_search = model_selection.GridSearchCV(k_neighbors, param_grid=params, cv=5, n_jobs=-1)
grid_search.fit(X_train_ageclass, y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'algorithm': ['ball_tree', 'kd_tree', 'brute'],
                         'n_neighbors': range(5, 15),
                         'weights': ['uniform', 'distance']})

In [162]:
grid_search.best_params_, grid_search.best_score_

({'algorithm': 'ball_tree', 'n_neighbors': 14, 'weights': 'uniform'},
 0.8193145439708743)

#### Perceptron

In [163]:
perceptron = linear_model.Perceptron()
params = {'penalty': ['l1', 'l2', 'elasticnet'],
          'alpha': np.arange(0.0001, 0.0005, 0.0001)}

grid_search = model_selection.GridSearchCV(perceptron, param_grid=params, cv=5, n_jobs=-1)
grid_search.fit(X_train_ageclass, y_train)

GridSearchCV(cv=5, estimator=Perceptron(), n_jobs=-1,
             param_grid={'alpha': array([0.0001, 0.0002, 0.0003, 0.0004]),
                         'penalty': ['l1', 'l2', 'elasticnet']})

In [164]:
grid_search.best_params_, grid_search.best_score_

({'alpha': 0.0004, 'penalty': 'l1'}, 0.7856004017324713)

#### LogisticRegression

In [165]:
logistic_regression = linear_model.LogisticRegression()
params = {'penalty': ['l1', 'l2', 'elasticnet', 'none'],
          'dual': [True, False],
          'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
          'multi_class': ['ovr', 'multinomial']}

grid_search = model_selection.GridSearchCV(logistic_regression, param_grid=params, cv=5, n_jobs=-1)
grid_search.fit(X_train_ageclass, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
295 fits failed out of a total of 400.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 447, in _check_solver
    raise ValueError(
Va

GridSearchCV(cv=5, estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'dual': [True, False],
                         'multi_class': ['ovr', 'multinomial'],
                         'penalty': ['l1', 'l2', 'elasticnet', 'none'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                                    'saga']})

In [166]:
grid_search.best_params_, grid_search.best_score_

({'dual': False, 'multi_class': 'ovr', 'penalty': 'l1', 'solver': 'liblinear'},
 0.7878852551628899)

#### SGDClassifier

In [167]:
sgd_classifier = linear_model.SGDClassifier()
params = {'loss': ['hinge', 'squared_hinge', 'log', 'squared_error'],
          'penalty': ['l1', 'l2', 'elasticnet']}

grid_search = model_selection.GridSearchCV(sgd_classifier, param_grid=params, cv=5, n_jobs=-1)
grid_search.fit(X_train_ageclass, y_train)

GridSearchCV(cv=5, estimator=SGDClassifier(), n_jobs=-1,
             param_grid={'loss': ['hinge', 'squared_hinge', 'log',
                                  'squared_error'],
                         'penalty': ['l1', 'l2', 'elasticnet']})

In [168]:
grid_search.best_params_, grid_search.best_score_

({'loss': 'log', 'penalty': 'elasticnet'}, 0.7889900194589166)

#### LinearSVC

In [169]:
linear_svc = LinearSVC()
params = {'penalty': ['l1', 'l2'],
          'loss': ['hinge', 'squared_hinge'],
          'dual': [True, False],
          'multi_class': ['ovr', 'crammer_singer'],
          'class_weight': [None, 'balanced']}

grid_search = model_selection.GridSearchCV(linear_svc, param_grid=params, cv=5, n_jobs=-1)
grid_search.fit(X_train_ageclass, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
40 fits failed out of a total of 160.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_classes.py", line 257, in fit
    self.coef_, self.intercept_, self.n_iter_ = _fit_liblinear(
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py", line 1185, in _fit_liblinear
    solver_type = _get_liblinear_solver_type(mul

GridSearchCV(cv=5, estimator=LinearSVC(), n_jobs=-1,
             param_grid={'class_weight': [None, 'balanced'],
                         'dual': [True, False],
                         'loss': ['hinge', 'squared_hinge'],
                         'multi_class': ['ovr', 'crammer_singer'],
                         'penalty': ['l1', 'l2']})

In [170]:
grid_search.best_params_, grid_search.best_score_

({'class_weight': 'balanced',
  'dual': True,
  'loss': 'hinge',
  'multi_class': 'crammer_singer',
  'penalty': 'l2'},
 0.792348251836043)

#### GaussianNB

In [171]:
gaussian = GaussianNB()
gaussian.fit(X_train_ageclass, y_train)
gaussian.score(X_train_ageclass, y_train)

0.7867564534231201

#### XGBClassifier

In [9]:
xgb_cl = XGBClassifier()
params = {'n_estimators': range(1, 11),
          'max_depth': range(1, 31),
          'learning_rate': np.arange(0.1, 1, 0.1),
          'verbosity': range(1, 4),
          'booster': ['gbtree', 'gblinear', 'dart']}

grid_search = model_selection.GridSearchCV(xgb_cl, param_grid=params, cv=5, n_jobs=-1)
grid_search.fit(X_train_ageclass, y_train)

[19:19:58] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     enable_categorical=False, gamma=None,
                                     gpu_id=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,...
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, reg_alpha=None,
                                     reg_lambda=None, scale_pos_weight=None,
                                     subsample=None, tree_

In [20]:
grid_search.best_params_, grid_search.best_score_

({'booster': 'dart',
  'learning_rate': 0.9,
  'max_depth': 5,
  'n_estimators': 6,
  'verbosity': 1},
 0.4200882491918046)

#### XGBRFClassifier

In [23]:
xgbrf_cl = XGBRFClassifier()
params = {'n_estimators': range(1, 11),
          'max_depth': range(1, 31),
          'learning_rate': np.arange(0.1, 1, 0.1),
          'verbosity': range(1, 4),
          'booster': ['gbtree', 'gblinear', 'dart']}

grid_search = model_selection.GridSearchCV(xgbrf_cl, param_grid=params, cv=5, n_jobs=-1)
grid_search.fit(X_train_ageclass, y_train)

[20:10:11] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


GridSearchCV(cv=5,
             estimator=XGBRFClassifier(base_score=None, booster=None,
                                       colsample_bylevel=None,
                                       colsample_bytree=None,
                                       enable_categorical=False, gamma=None,
                                       gpu_id=None, importance_type=None,
                                       interaction_constraints=None,
                                       max_delta_step=None, max_depth=None,
                                       min_child_weight=None, missing=nan,
                                       monotone_constraints=None,
                                       n_estimators=100, n_jobs=None,
                                       num_parall...
                                       objective='binary:logistic',
                                       predictor=None, random_state=None,
                                       reg_alpha=None, scale_pos_weight=None,
      

In [24]:
grid_search.best_params_, grid_search.best_score_

({'booster': 'gbtree',
  'learning_rate': 0.1,
  'max_depth': 4,
  'n_estimators': 3,
  'verbosity': 1},
 0.8204318624066287)